In [ ]:
import numpy as np
import random
import torch
import torch.utils.data as data
from PIL import Image
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

import os
import json
import os.path
import colorsys


inf = float('inf')
nan = float('nan')

In [ ]:
data_dir = "/home/aravind/dataset/"
ann_dir = data_dir + "annotations/panoptic/"
cat_file = ann_dir + "panoptic_coco_categories.json"

train_img_dir = data_dir + "train2017/"
train_seg_dir = ann_dir + "panoptic_train2017/"
train_ann_json = ann_dir + "panoptic_train2017.json"

val_img_dir = data_dir + "val2017/"
val_seg_dir = ann_dir + "panoptic_val2017/"
val_ann_json = ann_dir + "panoptic_val2017.json"

In [ ]:
with open(val_ann_json,"r") as f:
    val_ann = json.load(f)
with open(train_ann_json,"r") as f:
    train_ann = json.load(f)
with open(cat_file,"r") as f:
    cat_list = json.load(f)
coco_cats = {c['id']: c for c in cat_list}

In [ ]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 16
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 16
    PIN_MEMORY = True
    VALIDATION_STEPS = 20
    # including bg
    NUM_CLASSES = 81

    MEAN_PIXEL = np.array(
        [0.485, 0.456, 0.406], dtype=np.float32).reshape(1, 1, -1)
    STD_PIXEL = np.array(
        [0.229, 0.224, 0.225], dtype=np.float32).reshape(1, 1, -1)
    GRID_SHAPE = 6
    IMPULSE_SHAPE = (32, 32)
    MIN_PIXELS = 1
    MIN_INTERSECTION = 1
    def __init__(self):
        self.WIDTH = 32 * self.GRID_SHAPE
        self.HEIGHT = 32 * self.GRID_SHAPE
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT, 3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [ ]:
class CocoDetection(data.Dataset):
    def __init__(self, img_dir, ann_file, config):
        self.img_dir = img_dir
        self.coco = self.index_coco(ann_file)
        self.config = config
        self.catMap = self.build_class_map()



In [ ]:
def rgb2id(color):
    return color[:, :, 0] + 256 * color[:, :, 1] + 256 * 256 * color[:, :, 2]

In [ ]:
# print(val_ann['annotations'][0].keys())
# print(val_ann.keys())
# print(val_ann['categories'][:3])
# print(coco_cats)
# print(val_ann['info'])
# print(val_ann['annotations'][0]['image_id'])
# print(val_ann["annotations"][0])
# print(val_ann['images'])
ann = val_ann["annotations"][0]
Image.open(val_dir+ann['file_name'].replace('.png','.jpg')).show()
# Image.open(val_seg+ann['file_name']).show()
# print(coco_cats[23])
segmentation = np.array(Image.open(val_seg+ann['file_name']),dtype=np.uint8)
segmentation_id = rgb2id(segmentation)
for segment_info in ann['segments_info']:
    mask = (segmentation_id==segment_info['id'])
    Image.fromarray((mask*255).astype(np.uint8),"L").show()
print(val_ann['images'][0])

In [ ]:
# ann = np.random.choice(val_ann['annotations'])
Image.open()
for i in range(len(val_ann['annotations'])):
    ann = val_ann['annotations'][i]
    image_info = val_ann["images"][i]
    print(image_info['id'])
    print(ann['image_id'])
    print(i)
# for image_info in val_ann["images"]:
#     if image_info['id'] == ann['image_id']:
#         print(image_info['id'])